# Evaluación de Modelos con Conjunto de Datos Propio

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00


## Paso 1: Cargar las Librerías

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from datasets import load_dataset
import evaluate
from tqdm import tqdm
import json
import requests

In [4]:
# URLs of the files
urls = {
    "eval_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_mexico_dataset.json",
    "train_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_mexico_dataset.json",
    "train_colombia_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_dataset.json",
    "train_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_mexico_dataset.json",
    "eval_colombia_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_dataset.json",
    "eval_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_mexico_dataset.json",
}

# Dictionary to store the loaded JSON data
datasets = {}

for filename, url in urls.items():
    # Download the file
    response = requests.get(url)
    if response.status_code == 200:
        # Save locally
        with open(filename, "wb") as f:
            f.write(response.content)
        # Load JSON into Python
        datasets[filename] = response.json()
        print(f"{filename} downloaded and loaded successfully!")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

eval_colombia_mexico_dataset.json downloaded and loaded successfully!
train_colombia_mexico_dataset.json downloaded and loaded successfully!
train_colombia_dataset.json downloaded and loaded successfully!
train_mexico_dataset.json downloaded and loaded successfully!
eval_colombia_dataset.json downloaded and loaded successfully!
eval_mexico_dataset.json downloaded and loaded successfully!


## Paso 2: Cargar los modelos

In [5]:
models = {
    "BERT (Base)": "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    "DistilBERT (Base)": "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    "BERT (Fine-tuned News QA)": "BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    "DistilBERT (Fine-tuned News QA)": "BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
}

## Paso 3: Cargar los conjuntos de datos

In [6]:
GOLD_PATH   = "" #"../datasets/exploration_datasets/gold/"

In [7]:
datasets = {
    "Colombia and Mexico Train": GOLD_PATH + "train_colombia_mexico_dataset.json",
    "Colombia and Mexico Eval": GOLD_PATH + "eval_colombia_mexico_dataset.json",
    "Colombia Train": GOLD_PATH + "train_colombia_dataset.json",
    "Mexico Train": GOLD_PATH + "train_mexico_dataset.json",
    "Colombia Eval": GOLD_PATH + "eval_colombia_dataset.json",
    "Mexico Eval": GOLD_PATH + "eval_mexico_dataset.json"
}

## Paso 4: Cargar las métricas

In [8]:
# 🧮 Metric (SQuAD v2)
metric = evaluate.load("squad_v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


## Paso 5: Transformar el conjunto de datos a español: Aplanar a formato Contexto + Preguntas y Respuestas (QA

In [9]:
def flatten_squad(dataset):
    # Handle "data" key if present
    if isinstance(dataset, dict) and "data" in dataset:
        dataset = dataset["data"]

    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "title": article.get("title", ""),
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

## Paso 5: Evaluar el Modelo

In [10]:
def evaluate_model(model_name, model_path, dataset_dict):
    print(f"\n🔹 Evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

    for ds_name, ds_path in dataset_dict.items():
        print(f"\n📘 Dataset: {ds_name}")
        dataset = json.load(open(ds_path, "r", encoding="utf-8"))
        flat_eval = flatten_squad(dataset)

        predictions, references = [], []

        for ex in tqdm(flat_eval):
            for qa in ex["qas"]:
                if qa.get("is_impossible", False):
                    predictions.append({
                        "id": qa["id"],
                        "prediction_text": "",
                        "no_answer_probability": 1.0
                    })
                    references.append({
                        "id": qa["id"],
                        "answers": {"text": [], "answer_start": []}
                    })
                else:
                    try:
                        pred = qa_pipeline(
                            question=qa["question"],
                            context=ex["context"],
                            handle_impossible_answer=True
                        )
                        predictions.append({
                            "id": qa["id"],
                            "prediction_text": pred["answer"],
                            "no_answer_probability": 1.0 - pred.get("score", 0.0)
                        })
                        references.append({
                            "id": qa["id"],
                            "answers": {
                                "text": [a["text"] for a in qa["answers"]],
                                "answer_start": [a["answer_start"] for a in qa["answers"]]
                            }
                        })
                    except Exception as e:
                        print(f"⚠️ Skipped QA {qa.get('id', 'unknown')}: {e}")

        results = metric.compute(predictions=predictions, references=references)
        print(f"\n✅ Results for {ds_name}:")
        print(results)  # 👈 useful to confirm the keys
        print(f"   • Exact Match (EM): {results.get('exact_match', results.get('exact', 0)):.2f}")
        print(f"   • F1 Score: {results.get('f1', 0):.2f}")

## Ejecutar para ambos modelos

In [11]:
# 🚀 Run for both models
for model_name, model_path in models.items():
    evaluate_model(model_name, model_path, datasets)


🔹 Evaluating BERT (Base)


tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...


✅ Results for Mexico Train:
{'exact': 98.09898955300565, 'f1': 98.73033984922, 'total': 5839, 'HasAns_exact': 97.2646623952686, 'HasAns_f1': 98.17310359280324, 'HasAns_total': 4058, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 1781, 'best_exact': 98.09898955300565, 'best_exact_thresh': 0.9779223203659058, 'best_f1': 98.73033984922002, 'best_f1_thresh': 0.9999884366989136}
   • Exact Match (EM): 98.10
   • F1 Score: 98.73

📘 Dataset: Colombia Eval


100%|██████████| 2690/2690 [04:49<00:00,  9.29it/s]



✅ Results for Colombia Eval:
{'exact': 78.71431013848522, 'f1': 83.9610608989761, 'total': 5849, 'HasAns_exact': 69.22886801779535, 'HasAns_f1': 76.8137037068985, 'HasAns_total': 4046, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 1803, 'best_exact': 78.71431013848522, 'best_exact_thresh': 0.9999402165412903, 'best_f1': 83.96106089897656, 'best_f1_thresh': 0.9999915957450867}
   • Exact Match (EM): 78.71
   • F1 Score: 83.96

📘 Dataset: Mexico Eval


100%|██████████| 1156/1156 [03:02<00:00,  6.34it/s]



✅ Results for Mexico Eval:
{'exact': 79.10507391130643, 'f1': 84.04854310169175, 'total': 2503, 'HasAns_exact': 69.68115942028986, 'HasAns_f1': 76.85420486001995, 'HasAns_total': 1725, 'NoAns_exact': 100.0, 'NoAns_f1': 100.0, 'NoAns_total': 778, 'best_exact': 79.10507391130643, 'best_exact_thresh': 0.9972954988479614, 'best_f1': 84.04854310169178, 'best_f1_thresh': 0.9999886155128479}
   • Exact Match (EM): 79.11
   • F1 Score: 84.05
